# Ударим дропаутом по переобучению!

## Проблема переобучения

$$P_n(x) = \sum_{k=0}^n \alpha_k x^k$$
$$L = \sum_{i=0}^N (P_n(x) - y_i)^2$$

$n$ - степерь полинома, $N$ - количество наблюдений


Приближаем данные полиномом степени $n$ 
- n=0 - горизонтальная прямая (ср.арифм. или мат.ожидаение)
- n=1 - линейная регрессия
- ...
- n=N - линия пройдет в точности через все точки наблюдения (если $L\to min$, $L=0$) 
  - в этом случае говорят, что модель "выучила" обучающую выборку, но полученный полином совершенно не будет отражать фактическую зависимость в данных ($x$ от $y$)

Когда у модели много параметров - она имеет тенденцию к переобучению, которого нужно избегать

### Направления борьбы с переобучением

1. Не усложнять модель больше чем нужно
2. Увеличить количество данных
3. Сравнение качества обучения с валидационной выборкой
4. Регуляризация
5. Дропаут

Увеличение количества данных на основе имеющихся данных называется **аугментацией**
- например, в случае изображений - создание измененных кадров на основе имеющихся
  - геометрические преобразования (вырезания фаргментов, повороты, сжатия, отражения)
  - изменения цвета, контрастности и т.п.
  - добавление шумов
    - шум "соль-перец" (случайно возникающие чёрные и белые пиксели в кадре)
    - гауссовский шум
  - разнообразные другие эффекты, сохраняющие исходный класс изображения
- в валидационную выборку не должны попадать такие дополнительные данные

Сравнение качества обучения с валидационной выборкой - 
- "раннее завершение" (early stopping) - остановка обучения, когда функция потерь на валидационной выборке начала расти
  - как вариант - сохранять контрольные точки и по итогу выбирать состояние с минимальными потерями на валидации

### Регуляризация

Изменение функции потерь с добавленим штрафа за слишком большие коэффициенты (веса сети, к-ты полинома)

#### **Регуляризация Тихонова** ($L_2$-регуляризация / weight decay / ridge regression)

$$\tilde L = L + \lambda \sum_{k=0}^{\#\alpha} \alpha_k^2$$

- weight decay - это похожее, но не совсем это

Проблема регуляризации - изменяется функция потерь и в результате оптимум ищется не совсем от нужного функционала. Есть значимая вероятность попасть в локальный минимум регуляризованной функции, который будет соответствовать пику исходной функции. 

**Пример**: $L(\alpha) = cos(\alpha) + \lambda \alpha^2$ - где периодический косинус представляет истинную функцию потерь, у которой много локальных минимумов, один из которых мы хотим найти. При "больших" $\lambda$ парабола регулярзации "продавит" косинус так, что останутся только два варианта из всех возможных $\{-\pi, +\pi\}$. Если увеличить $\lambda$ еще больше, то может получиться, что минимум окажется в 0, что вообще не соответствует реальному минимуму функции потерь.

`AdamW` - это реализация `Adam` с применением **Weight Decay**. 

Что такое Weight Decay? При каждом обновлении веса давайте кроме движения в сторону антиградиента еще и будем вычитать маленький кусочек веса, умноженный на некоторый гиперпараметр. Например, формула стохастического градиентного спуска с применением Weight Decay будет выглядеть так:

$w = w - lr * \dfrac{\partial L}{\partial w} - lr * wd * w$ 

Здесь lr - learning rate, wd - weight decay, гиперпараметр.

Если Вы внимательно посмотрите на эту формулу, то заметите, что в случае стохастического градиентного спуска Weight Decay эквивалентен применению L2-регуляризации к loss-функции:

$$L_{new} = L + \dfrac{wd}{2} ||w||^2 \\
 
\dfrac{\partial (L_{new})}{\partial w} = \dfrac{\partial L}{\partial w} + wd \cdot w \\

w = w - lr \cdot \dfrac{\partial L_{new}}{\partial w} = w - lr * \dfrac{\partial L}{\partial w} - lr * wd * w $$

Однако, L2-регуляризация (**меняем loss-функцию**) и Weight Decay (**не меняем loss-функцию**, меняем только способ обновления весов) работают одинаково только в простом случае стохастического градиентного спуска, в случае адаптивных оптимизаторов, например, Адама, эти два подхода различаются (причем эмпирически было показано, что часто Weight Decay работает лучше). Вся эта история породила некоторую путаницу в терминилогии, и во многих библиотеках Адам реализовали именно с применением L2-регуляризации, ошибочно называя такой подход Weight Decay.  Авторы статьи [Decoupled Weight Decay Regularization](https://arxiv.org/abs/1711.05101) решили разграничить **Adam+L2** и **Adam+Weight Decay**, назвав последнее AdamW.


#### **Регуляризация LASSO** ($L_1$-регуляризация)

$$\tilde L = L + \lambda \sum_{k=0}^{\#\alpha} | \alpha_k|$$

Линии уровня слагаемого регуляризации теперь не окружности как в $L_2$, а ромбы (размерности $\#\alpha$ конечно)
- значит в сумме с исходной функцией потерь, минимум будет где-то на осях этих линий уровня, т.е. лассо-регуляризация "прореживает" параметры модели, зануляет менее значимые параметры
- регуляризация Тихонова диффиренцируема в любой точке, LASSO - нет
- по модулю производная LASSO всегда равна $\lambda$, если она дифференцируема

### Дропаут

Вид регуляризации, характерный только для нейронных сетей.

1 вариант (drop connection) - с вероятностью $p$ пропадает связь между нейронами ($w=0$)

2 вариант (drop neuron) - с вероятностью $p$ нейрон перестает выдавать сигнал

# Батч-нормализация

Нормализованные данные приводят к более быстрому обучению модели

Если данные сильно смещены от начала координат, то потребуется много шагов, чтобы изначальная разделяющая поверхность, которая появилась в результате случайной инициализации, а значит где-то вокруг 0, дойдет до разделения удаленных в некотую сторону от 0 данных.

Вытянутая вдоль какой-то оси форма данных также приводит вытягиванию функции потерь вдоль этой оси, появления оврагов, где схождение происходит долго.

**Нормировка** - заключается в том, чтобы вычесть матожидание и поделить данные на стандартные отклонения по осям.
- если стандартные отклонения одинаковые, то овраг останется, т.е. данные коррелируют
- но если разделяющая поверхность проходит через математическое ожидание всей тренировочной выборки, то после нормализации она будет проходить через 0, и значит не нужно будет искать смещение

$$\vec {\tilde x} = \frac {\vec x - \vec \mu}{\vec \sigma} \\
\vec \mu = \frac{\sum \vec {x_i}}{N} \\
\vec \sigma = \sqrt {\frac {\sum (\vec x_i - \vec \mu)^2}{N-1}}$$

Если мы обучаем по батчам, то это преобразование делается для каждого батча:

$$\vec {\tilde x} = \frac {\vec x - \vec \mu_B}{\vec \sigma_B}\cdot \gamma + \beta \\
\vec \mu_B = \frac{\sum \vec {x_i}}{N_B} \\
\vec \sigma_B = \sqrt {\frac {\sum (\vec x_i - \vec \mu_B)^2}{N_B-1}}$$

$\gamma$ - обучаемый параметр, отвечает за СКО нейрона

$\beta$ - обучаемый параметр, отвечает за МО нейрона

$\gamma$, $\beta$ - изменяются от батча к батчу, а статистика по МО и СКО данных накапливается в отдельных параметрах, которые и используются на этапе генерации ответов обученной сети:

$$\hat {\vec \mu} = EMA \vec \mu_B\\
\hat {\vec \sigma} = EMA \sigma_B$$

- скользящие экспоненциальные средние 
- с их помощью преобразуются (нормализуются) входные данные на тесте/валидации

Почему мы не можем в процессе обучения заменить статистики, подсчитанные по батчу, на статистики, которые подсчитаны при помощи скользящих экспоненциальных средних? Здесь причина заключается вот в чём: когда мы рассматриваем статистики по батчу, то мы можем по каждой статистике сделать обратное распространение ошибки. Если бы она зависила от параметров, являющихся некоторой функцией предыдущих состояний, то нам надо было бы по ним по всем брать градиент. Смысл тогда батчевать выборку?

**Очень эффективный прием обучения** - это очень легкая операция, которая практически не потребляет ни памяти, ни вычислительного времени.

Куда можно добавить:

- если свертка - вставляете батч-нормализацию двухмерную, 
- если полносвязанный слой - вставляйте одномерную батч-нормализацию.

Куда нельзя добавить:

- туда, где она не нужна
  - уже центрированные данные
  - сложные данные вроде текстовых эмбеддингов, т.е. где информация структурирована сложнее чем яркость пикселя в канале больше / меньше, например представляет собой вектор, который непонятно как нормализовывать не зная всех превсех вообще остальных векторов

# Семинар: cлой нормализации

Самая популярная версия слоя нормализации - слой нормализации "по батчу" (batch-norm слой).

Рассмотрим его работу в наиболее простом случае, когда на вход подается батч из одномерных векторов:

На вход подается батч одномерных векторов:

$$x_i^{(j)}$$

где $j$ индекс вектора внутри батча, $i$ - номер компоненты.

Для текущего батча:

- По каждой компоненте входа вычисляются мат.ожидание и дисперсия:

$$E(x_i) = \frac{\sum_{j=1}^N x_i^{(j)}}{N} \\
\sigma (x_i)^2 = \frac{\sum_{j=1}^N (x_i^{(j)} - E(x_i))_j^2}{N}$$


- Вход нормируется по формуле: 

$$z_i^{(j)} = \frac{x_i^{(j)} - E(x_i)}{\sqrt {\sigma (x_i)^2 + \varepsilon}}$$

Эпсилон необходим для случая нулевой дисперсии.

- Нормированный вход преобразуется следующим образом:

$$y_i^{(j)} = z_i^{(j)} \gamma_i + \beta_i$$

Где **Гамма** и **Бета** - обучаемые параметры слоя. Обратите внимание, Гамма и Бета - вектора такой же длины, как инстансы входа.

Их можно фиксировать, например, простейший случай - Бета принимается равным нулевому вектору, Гамма - вектору из единиц. 

Если же взять Гамму равным знаменателю дроби из формулы для **Z**, а Бету равным мат.ожиданию, то слой вернет входной тензор без изменений. То есть, слой будет эквивалентен тождественной функции.

Таким образом, параметры Бета и Гамма позволяют не терять входящию в слой информацию, и одновременно с этим, батч-норм слой нормализует вход. Последнее ускоряет сходимость параметров сети, а в некоторых случаях без нормализации добиться сходимости сети крайне сложно.

Итоговая формула преобразования входа: 

$$y_i^{(j)} = \frac{x_i^{(j)} - E(x_i)}{\sqrt {\sigma (x_i)^2 + \varepsilon}} \gamma_i + \beta_i$$

В этом уроке мы будем двигаться по следующему плану:

- Вначале реализуем train-этап батч-нормализации для батча из одномерных векторов с нулевым Бета и единичным Гамма.
- Затем добавим возможность задания параметров Бета и Гамма.
- После этого добавим eval-этап использования слоя.
- И последним шагом по батч-нормализации реализуем train-этап слоя батч-нормализации для батча из многоканальных двумерных тензоров с нулевым Бета и единичным Гамма.

В данном шаге вам требуется реализовать функцию батч-нормализации без использования стандартной функции со следующими упрощениями:

- Параметр Бета принимается равным 0.
- Параметр Гамма принимается равным 1.
- Функция должна корректно работать только на этапе обучения.
- Вход имеет размерность число элементов в батче * длина каждого инстанса.

In [46]:
import numpy as np
import torch
import torch.nn as nn

def custom_batch_norm1d(input_tensor, eps):
    # ex = input_tensor.mean(dim=0)
    # sd = (((input_tensor - ex) ** 2).mean(dim=0) + eps).sqrt()
    # normed_tensor = (input_tensor - ex) / sd

    normed_tensor = ((input_tensor - input_tensor.mean(dim=0)) / 
                     torch.sqrt(input_tensor.var(dim=0, unbiased=False) + eps))
    return normed_tensor

input_tensor = torch.Tensor([[0.0, 0, 1, 0, 2], [0, 1, 1, 0, 10]])
batch_norm = nn.BatchNorm1d(input_tensor.shape[1], affine=False)

import numpy as np
all_correct = True
for eps_power in range(10):
    eps = np.power(10., -eps_power)
    batch_norm.eps = eps
    batch_norm_out = batch_norm(input_tensor)
    custom_batch_norm_out = custom_batch_norm1d(input_tensor, eps)

    all_correct &= torch.allclose(batch_norm_out, custom_batch_norm_out)
    all_correct &= batch_norm_out.shape == custom_batch_norm_out.shape
print(all_correct)

True


Немного обобщим функцию с предыдущего шага - добавим возможность задавать параметры Бета и Гамма.

На данном шаге вам требуется реализовать функцию **батч-нормализации без использования стандартной функции** со следующими упрощениями:

- Функция должна корректно работать только на этапе обучения.
- Вход имеет размерность число элементов в батче * длина каждого инстанса.

In [47]:
import torch
import torch.nn as nn

input_size = 7
batch_size = 5
input_tensor = torch.randn(batch_size, input_size, dtype=torch.float)

eps = 1e-3

def custom_batch_norm1d(input_tensor, weight, bias, eps):
    normed_tensor = ((input_tensor - input_tensor.mean(dim=0)) / 
                     torch.sqrt(input_tensor.var(dim=0, unbiased=False) + eps))
    return normed_tensor * weight + bias
    

batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm_out = batch_norm(input_tensor)
custom_batch_norm_out = custom_batch_norm1d(input_tensor, batch_norm.weight.data, batch_norm.bias.data, eps)
print(torch.allclose(batch_norm_out, custom_batch_norm_out) \
      and batch_norm_out.shape == custom_batch_norm_out.shape)

True


Избавимся еще от одного упрощения - реализуем работу слоя батч-нормализации на этапе предсказания.

На этом этапе вместо статистик по батчу будем использовать экспоненциально сглаженные статистики из истории обучения слоя.

В данном шаге вам требуется реализовать полноценный класс батч-нормализации без использования стандартной функции, принимающий на вход двумерный тензор. Осторожно, расчёт дисперсии ведётся по смещенной выборке, а расчет скользящего среднего по несмещенной.

In [70]:
import torch
import torch.nn as nn


input_size = 3
batch_size = 5
eps = 1e-1


class CustomBatchNorm1d:
    def __init__(self, weight, bias, eps, momentum):
        self.weight = weight
        self.bias =  bias 
        self.eps = eps
        self.momentum = momentum
        self.ema_e =  0
        self.ema_sd = 1
        self.train = True

    def __call__(self, input_tensor):
        if self.train:
            e_next = input_tensor.mean(dim=0)
            sd_next = input_tensor.var(dim=0, unbiased=False)
            ololo = input_tensor.shape[0] / (input_tensor.shape[0] - 1)
            self.ema_e  = (1 - self.momentum)*e_next + self.momentum*self.ema_e
            self.ema_sd = (1 - self.momentum)*sd_next*ololo  + self.momentum*self.ema_sd
        else:
            e_next, sd_next = self.ema_e, self.ema_sd 

        normed_tensor = (input_tensor - e_next) / torch.sqrt(sd_next + eps)
        return normed_tensor * self.weight + self.bias

    def eval(self):
        self.train = False


batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm.momentum = 0.5

custom_batch_norm1d = CustomBatchNorm1d(batch_norm.weight.data,
                                        batch_norm.bias.data, eps, batch_norm.momentum)

all_correct = True

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output, atol=1e-04) \
        and norm_output.shape == custom_output.shape

batch_norm.eval()
custom_batch_norm1d.eval()

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output, atol=1e-04) \
        and norm_output.shape == custom_output.shape
print(all_correct)

True


Слой батч-нормализации существует для входа любой размерности.

В данном шаге рассмотрим его для входа из многоканальных двумерных тензоров, например, изображений.

Если вытянуть каждый канал картинки в вектор, то вход будет трехмерным:

- количество картинок в батче
- число каналов в каждой картинке
- число пикселей в картинке

Процесс нормализации:

- Вход разбивается на срезы, параллельные синей части. То есть, каждый срез - это все пиксели всех изображений по одному из каналов.
- Для каждого среза считаются мат. ожидание и дисперсия.
- Каждый срез нормализуется независимо.
 

На данном шаге вам предлагается реализовать батч-норм слой для четырехмерного входа (например, батч из многоканальных двумерных картинок) без использования стандартной реализации со следующими упрощениями:

- Параметр Бета = 0.
- Параметр Гамма = 1.
- Функция должна корректно работать только на этапе обучения.

**По батчу**

In [195]:
import torch
import torch.nn as nn

eps = 1e-3

input_channels = 3
batch_size = 3
height = 10
width = 10

batch_norm_2d = nn.BatchNorm2d(input_channels, affine=False, eps=eps)

input_tensor = torch.randn(batch_size, input_channels, height, width, dtype=torch.float)


def custom_batch_norm2d(input_tensor, eps):
    shape = input_tensor.shape
    input_tensor = input_tensor.transpose(1, 0).flatten(1)  # измерение батча к кадру
    normed_tensor = (input_tensor - input_tensor.mean(dim=1).unsqueeze(-1)) / \
                    torch.sqrt(input_tensor.var(dim=1, unbiased=False).unsqueeze(-1) + eps)
    return normed_tensor.reshape(shape).transpose(1, 0)


norm_output = batch_norm_2d(input_tensor)
custom_output = custom_batch_norm2d(input_tensor, eps)
print(torch.allclose(norm_output, custom_output) and norm_output.shape == custom_output.shape)

True


Мы познакомились поближе с нормализацией "по батчу". Для упрощения дальнейшего изложения остановимся на случае трехмерного тензора на входе слоя, если же размерность входа больше трех, то вытянем все размерности кроме первых двух в одну размерность.

Бывает нормировка не только по батчу, но и по другим измерениям.

Обратите внимание на изображения ниже.

<очень информативное изображение> 

Где:

C - число каналов на входе.
N - размер батча.
H, W - размерность по последней (третьей) размерности входа.

На изображении можно увидеть следующие виды нормализации:

- По батчу.
- По каналу.
- По инстансу (эт типа вектор кадра в одном канале).
- По группе (эт типа несколько векторов кадра в разных каналов).

**По каналу**

In [203]:
eps = 1e-10

def custom_layer_norm(input_tensor, eps):
    """по каналу, значит нам просто не нужно переставлять измерения, 
    канал и так рядом с кадром"""
    shape = input_tensor.shape
    input_tensor = input_tensor.flatten(1)
    normed_tensor = (input_tensor - input_tensor.mean(dim=1).unsqueeze(-1)) / \
                    torch.sqrt(input_tensor.var(dim=1, unbiased=False).unsqueeze(-1) + eps)
    return normed_tensor.reshape(shape)

all_correct = True
for dim_count in range(3, 9):
    input_tensor = torch.randn(*list(range(3, dim_count + 2)), dtype=torch.float)
    layer_norm = nn.LayerNorm(input_tensor.size()[1:], elementwise_affine=False, eps=eps)

    norm_output = layer_norm(input_tensor)
    custom_output = custom_layer_norm(input_tensor, eps)

    all_correct &= torch.allclose(norm_output, custom_output, 1e-2)
    all_correct &= norm_output.shape == custom_output.shape
print(all_correct)

True


На этом шаге вам предлагается реализовать нормализацию "по инстансу" без использования стандартного слоя со следующими упрощениями:

- Параметр Бета = 0.
- Параметр Гамма = 1.
- На вход подается трехмерный тензор (размер батча, число каналов, длина каждого канала инстанса).
- Требуется реализация только этапа обучения.

**По инстансу**

In [202]:
eps = 1e-3

batch_size = 5
input_channels = 2
input_length = 30

instance_norm = nn.InstanceNorm1d(input_channels, affine=False, eps=eps)

input_tensor = torch.randn(batch_size, input_channels, input_length, dtype=torch.float)


def custom_instance_norm1d(input_tensor, eps):
    """тут одномерим только кадр канала, среднее и СКО по последним измерениям"""
    shape = input_tensor.shape
    input_tensor = input_tensor.flatten(2)
    normed_tensor = (input_tensor - input_tensor.mean(dim=2).unsqueeze(-1)) / \
                    torch.sqrt(input_tensor.var(dim=2, unbiased=False).unsqueeze(-1) + eps)
    return normed_tensor.reshape(shape)

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
norm_output = instance_norm(input_tensor)
custom_output = custom_instance_norm1d(input_tensor, eps)
print(torch.allclose(norm_output, custom_output, atol=1e-06) and norm_output.shape == custom_output.shape)

True


Нормализация "по группе" - это обобщение нормализации "по каналу" и "по инстансу".

Каналы в изображении не являются полностью независимыми, поэтому возможность использования статистики соседних каналов является преимуществом нормализации "по группе" по сравнению с нормализацией "по инстансу".

В то же время, каналы изображения могут сильно отличатся, поэтому нормализация "по группе" является более гибкой, чем нормализация "по каналу".

На этом шаге вам предлагается реализовать нормализацию "по группе" без использования стандартного слоя со следующими упрощениями:

- Параметр Бета = 0.
- Параметр Гамма = 1.
- Требуется реализация только этапа обучения.
- На вход подается трехмерный тензор.

**По группе**

In [253]:
import torch
import torch.nn as nn

channel_count = 6
eps = 1e-3
batch_size = 6
input_size = 2

input_tensor = torch.randn(batch_size, channel_count, input_size)


def custom_group_norm(input_tensor, groups, eps):
    """Expected number of channels in input to be divisible by num_groups
    groups - это несколько соседних каналов в штуках
    https://arxiv.org/pdf/1803.08494.pdf"""

    N, C, input_size = input_tensor.shape
    x = input_tensor.reshape(N, groups, C // groups * input_size)

    mean = x.mean(dim=2).unsqueeze(-1)
    var = x.var(dim=2, unbiased=False).unsqueeze(-1)
    x = (x - mean) / torch.sqrt(var + eps)
    x = x.reshape(N, C, input_size)
    return x 


    shape = input_tensor.shape
    input_tensor = input_tensor.flatten(2)
    normed_tensor = (input_tensor - input_tensor.mean(dim=2).unsqueeze(-1)) / \
                    torch.sqrt(input_tensor.var(dim=2, unbiased=False).unsqueeze(-1) + eps)
    return normed_tensor.reshape(shape)

all_correct = True
for groups in [1, 2, 3, 6]:
    group_norm = nn.GroupNorm(groups, channel_count, eps=eps, affine=False)
    norm_output = group_norm(input_tensor)
    custom_output = custom_group_norm(input_tensor, groups, eps)
    all_correct &= torch.allclose(norm_output, custom_output, 1e-3)
    all_correct &= norm_output.shape == custom_output.shape
print(all_correct)

True


# Теоретические задачи: регуляризация

**Снова рассмотрим задачу:**

$$Xa = y$$

где нужно найти вектор $\vec a$ размера $N$, $X$ и $y$ - известны, $X$ - матрица размером $M \times N$, $y$ - вектор размера $M$.

Как мы уже видели, данную задачу можно представить в виде задачи оптимизации $L = \|Xa - y\| \rightarrow min_{a}$.

Как мы помним, как задача оптимизации, так и линейное уравнение, имеют бесконечное количество решений, если матрица $X^T$ имеет хотя бы одно нулевое собственное значение.

С точки зрения оптимизационного процесса, все решения этой задачи оптимизации хороши, однако с точки зрения здравого смысла, слишком большие значения в векторе $\vec a$ не нужны, если имеется решение с меньшими весами.

Поэтому добавим регуляризацию в приведенную выше задачу оптимизации $L^* = \|Xa - y\|^2_2 + \lambda \|a\|^2_2 \rightarrow \min_{a}$.

**Запишите производную функции потерь** по вектору $\vec a$, $\frac {\partial L^*}{\partial a}$. Ответ представьте в матричном виде:

$$\frac{\partial L^*}{\partial a} = X^T \cdot 2 (Xa - y) + \lambda 2 a$$

**Запишите необходимое условие минимума для задачи оптимизации**

$$L^* = \|Xw - y\|^2_2 + \lambda \|w\|^2_2 \rightarrow \min_{w}$$

**Сколько решений** имеет эта задача (при фиксированном $\lambda > 0$)?

- 1

**Запишите решение задачи оптимизации в матричном виде**

$$\| Xa - y \|_2^2 + \lambda \|w\|^2_2 \rightarrow \min_{w} \\
\frac{\partial L^*}{\partial w} = X^T \cdot 2 (Xw - y) + 2 \lambda w= 0 \\
... \\
\vec w = (X^T X + \lambda E)^{-1}X^T \vec y$$

*) Такая регрессия называется гребневой регрессией (ridge regression). А гребнем является как раз диагональная матрица, которую мы прибавляем.

Рассмотрим задачу оптимизации:
$$\frac{1}{2}(wx - b)^2 + \lambda \left| w \right| \rightarrow \min_{w}$$

где $x$, $b$ - действительные числа, $\lambda$ - неотрицательное число.

Меньше какого значения $\lambda$ данная задача имеет решение $w \neq 0$?

На пальцах примерно так
$$\to wx^2 - bx \pm \lambda = 0\\
w = (bx \pm \lambda) / x^2 \\
\lambda < |bx|$$

**Семинар**: Решаем задачу классификации на датасете CIFAR в `./Neural_Networks_and_CV`